In [1]:
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, initializers, activations
from tensorflow.keras.applications import resnet, resnet50
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dfpatient1 = pd.read_csv("patient1.csv")
dfpatient2 = pd.read_csv("patient2.csv")
dfpatient3 = pd.read_csv("patient3.csv")
dfpatient4 = pd.read_csv("patient4.csv")

df_list = [dfpatient1, dfpatient2, dfpatient3, dfpatient4]
df = pd.concat(df_list)

In [3]:
df_train = df[['Fz',	'Cz',	'Pz',	'Oz',	'P3',	'P4',	'PO7',	'PO8']]
df_targets = df[['y_stim']]

num_classes = len(np.unique(df_targets[['y_stim']]))
print(num_classes)

13


In [4]:
df_train.shape, df_targets.shape

((1390816, 8), (1390816, 1))

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler

In [6]:
from scipy import stats

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 200
STEP = 40

X_train, y_train = create_dataset(
    df_train[['Fz',	'Cz',	'Pz',	'Oz',	'P3',	'P4',	'PO7',	'PO8']][1801:244102],
    df_targets[1801:244102], TIME_STEPS, STEP)

X_test, y_test = create_dataset(
    df_train[['Fz',	'Cz',	'Pz',	'Oz',	'P3',	'P4',	'PO7',	'PO8']][244102:344102],
    df_targets[244102:344102], TIME_STEPS, STEP)

print(X_train.shape, y_train.shape)

(6053, 200, 8) (6053, 1)


In [7]:
#from sklearn.preprocessing import OneHotEncoder

'''This part can be utilized for predicting the y_values which indicated whether the patient was
visualizing or tested on non-target or target stimulus, in order to allow us to understand the reaction in EEG wavelengths'''

#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

#enc = enc.fit(y_train)
#y_train = enc.transform(y_train)
#y_test = enc.transform(y_test)

#y_train

In [8]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, input_shape=[X_train.shape[1], X_train.shape[2]])))
model.add(keras.layers.Dropout(rate =  0.5))
model.add(keras.layers.Dense(128, activation = 'sigmoid'))
model.add(keras.layers.Dense(y_train.shape[1], activation = 'sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [9]:
history = model.fit(X_train, y_train, epochs = 20,
batch_size = 32, validation_split = 0.3, shuffle = False)

Epoch 1/20
133/133 [==============================] - 15s 113ms/step - loss: 0.0000e+00 - acc: 0.9958 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/20
133/133 [==============================] - 19s 142ms/step - loss: 0.0000e+00 - acc: 0.9958 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/20
133/133 [==============================] - 19s 140ms/step - loss: 0.0000e+00 - acc: 0.9969 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/20
133/133 [==============================] - 30s 225ms/step - loss: 0.0000e+00 - acc: 0.9950 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/20
133/133 [==============================] - 30s 223ms/step - loss: 0.0000e+00 - acc: 0.9972 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 6/20
133/133 [==============================] - 30s 224ms/step - loss: 0.0000e+00 - acc: 0.9960 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/20
133/133 [==============================] - 29s 219ms/step - loss: 0.0000e+00 - acc: 0.9962 - val_loss: 0.0000e+00 - val_acc: 1.0000

In [10]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

78/78 [==============================] - 5s 67ms/step - loss: 0.0000e+00 - acc: 1.0000
